# AlphaKnit Phase 10: Dàn Hàng Trận Google Colab
Notebook này được thiết lập đặc biệt cho việc Huấn luyện mô hình Topology DAG của AlphaKnit trên Scale 50,000+ dữ liệu.

### Bước 1: Kích hoạt GPU & Mount Google Drive
Mô hình cần ít nhất GPU dòng T4 (Miễn phí) để train 100 epochs.

In [1]:
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Bước 2: Kéo Source Code từ GitHub
Bạn cần đẩy thư mục `Project_AlphaKnit` trên máy bạn lên một <a href="https://github.com/new">Repo Github ẩn (Private)</a>.

**Sửa lại dòng lệnh dưới:** Thay bằng đường link Repo Github của bạn (VD: `https://<your-token>@github.com/username/AlphaKnit.git`).

In [2]:
import os
# Lùi ra ổ đĩa gốc
os.chdir('/content')
# Xóa trắng Repo cũ
!rm -rf AlphaKnit
# Kéo Repo mới nhất đã có argparse
!git clone https://github.com/KiettranFNF002/AlphaKnit-Topology.git AlphaKnit
# Trỏ vào lại thư mục làm việc
os.chdir('/content/AlphaKnit')
print("Thư mục hiện tại:")
!pwd


Cloning into 'AlphaKnit'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 81 (delta 6), reused 75 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 3.35 MiB | 20.07 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Thư mục hiện tại:
/content/AlphaKnit


### Bước 3: Cài đặt & Cấu hình
Tải các thư viện Python cần thiết (bao gồm torch, numpy, etc.)

In [3]:
# Lọc bỏ các thư viện Windows-only và thực hiện cài đặt
!grep -vE "pywin32|pypiwin32|win32|pyhook" requirements.txt > req_colab.txt
!pip install -r req_colab.txt

import sys
sys.path.append('/content/AlphaKnit/src')
print("✅ PYTHONPATH Set & Environment Ready!")


grep: requirements.txt: binary file matches
✅ PYTHONPATH Set & Environment Ready!


### Bước 4: Tạo Symlink Lưu Data Lên Drive
50.000 Samples (dạng `.npy`) sẽ ngốn hàng chục GB dung lượng. Nếu rớt phiên của Colab, mảng này cũng bay màu.
Lệnh sau lập symlink kéo 2 thư mục nội dung cứng (`data` & `checkpoints`) thẳng đến Google Drive để dữ liệu tự động đồng bộ ngay lập tức và bền bỉ.

In [4]:
!mkdir -p /content/drive/MyDrive/AlphaKnit_Phase10/data
!mkdir -p /content/drive/MyDrive/AlphaKnit_Phase10/checkpoints

!rm -rf /content/AlphaKnit/data
!rm -rf /content/AlphaKnit/checkpoints

!ln -s /content/drive/MyDrive/AlphaKnit_Phase10/data /content/AlphaKnit/data
!ln -s /content/drive/MyDrive/AlphaKnit_Phase10/checkpoints /content/AlphaKnit/checkpoints

print("✅ Symlink Google Drive Attached!")

✅ Symlink Google Drive Attached!


### Bước 5: Scale Nóng Dữ Liệu Lên 50,000 Mẫu Mới Cáo
Thuật toán Data Generator V2 đã tốt 100%. Lần này chúng ta tạo 1 mẻ siêu khổng lồ cho Phase 10.

In [11]:
# Ép file code trên Colab đổi thành 50,000 mẫu và chạy
!sed -i 's/n_samples=1000/n_samples=50000/g' scripts/gen_phase9b_data.py
!python scripts/gen_phase9b_data.py


  Generated 5000/50000 samples...
  Generated 10000/50000 samples...
  Generated 15000/50000 samples...
  Generated 20000/50000 samples...
  Generated 25000/50000 samples...
  Generated 30000/50000 samples...
  Generated 35000/50000 samples...
  Generated 40000/50000 samples...
  Generated 45000/50000 samples...
  Generated 50000/50000 samples...

Done: 50000 samples saved to '/content/AlphaKnit/data/processed/dataset_phase9b_full'
Skipped (invalid): 0


### Bước 6: TRAINING CHÍNH THỨC - Đêm Không Mất Ngủ
Sức mạnh vô lượng đã trong tay! Ấn "Chạy"!

In [5]:
!env PYTHONPATH="src" python -m alphaknit.train --epochs 100 --run_name colab_p10_full_scale --dataset_dir data/processed/dataset_phase9b_full


Training on: cuda
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/AlphaKnit/src/alphaknit/train.py", line 529, in <module>
    train(
  File "/content/AlphaKnit/src/alphaknit/train.py", line 364, in train
    train_loader, val_loader = make_dataloaders(
                               ^^^^^^^^^^^^^^^^^
  File "/content/AlphaKnit/src/alphaknit/knitting_dataset.py", line 114, in make_dataloaders
    dataset = KnittingDataset(dataset_dir)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/AlphaKnit/src/alphaknit/knitting_dataset.py", line 36, in __init__
    f[:-5] for f in os.listdir(dataset_dir)
                    ^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 5] Input/output error: 'data/processed/dataset_phase9b_full'
